# 第３回

##  xpathを使ったウェブスクレイピング

In [1]:
from lxml import html

with open("./c3.html", 'r') as f:
    fstr = f.read()

# lxmlでHTMLを解析
tree = html.fromstring(fstr)

# XPathでニュースタイトルを取得（GoogleChrome上の）

# どちらでも取れるのを確認
#titles = tree.xpath('/html/body/div/text()')
titles = tree.xpath('//div/text()')

# 結果を出力
for title in titles:
    print(title)


こんにちは世界



In [2]:
#response.content

## クローリング

### APIの例（郵便番号検索API）

In [3]:
import requests

In [4]:
res = requests.get("https://zipcloud.ibsnet.co.jp/api/search", params={'zipcode': '8701124'})

In [5]:
resDic = res.json()

In [6]:
resDic

{'message': None,
 'results': [{'address1': '大分県',
   'address2': '大分市',
   'address3': '旦野原',
   'kana1': 'ｵｵｲﾀｹﾝ',
   'kana2': 'ｵｵｲﾀｼ',
   'kana3': 'ﾀﾞﾝﾉﾊﾙ',
   'prefcode': '44',
   'zipcode': '8701124'}],
 'status': 200}

In [7]:
adDic1 = resDic["results"][0]

In [8]:
print(f"{adDic1['address1']}{adDic1['address2']}{adDic1['address3']}")

大分県大分市旦野原


### クローリング演習

#### 食べログ飲食店データ取得

##### 1. 取得したいレビューのURLの取得

In [9]:
baseurl = "https://tabelog.com"
URL=baseurl+"/oita/A4402/A440202/44005660/dtlrvwlst/"

In [10]:
URL

'https://tabelog.com/oita/A4402/A440202/44005660/dtlrvwlst/'

In [11]:
res = requests.get(URL)

In [12]:
tree = html.fromstring(res.content)

In [13]:
eles = tree.xpath("//a[@class='rvw-item__title-target']")

In [14]:
reviewurls = []
for ele in eles:
    aurl = baseurl+ele.get("href")
    reviewurls.append(aurl)

In [15]:
set(reviewurls)

{'https://tabelog.com/oita/A4402/A440202/44005660/dtlrvwlst/B111965666/?use_type=0&smp=1',
 'https://tabelog.com/oita/A4402/A440202/44005660/dtlrvwlst/B382464585/?use_type=0&smp=1',
 'https://tabelog.com/oita/A4402/A440202/44005660/dtlrvwlst/B417580252/?use_type=0&smp=1',
 'https://tabelog.com/oita/A4402/A440202/44005660/dtlrvwlst/B440585631/?use_type=0&smp=1',
 'https://tabelog.com/oita/A4402/A440202/44005660/dtlrvwlst/B440868409/?use_type=0&smp=1',
 'https://tabelog.com/oita/A4402/A440202/44005660/dtlrvwlst/B453057681/?use_type=0&smp=1',
 'https://tabelog.com/oita/A4402/A440202/44005660/dtlrvwlst/B458471005/?use_type=0&smp=1',
 'https://tabelog.com/oita/A4402/A440202/44005660/dtlrvwlst/B478849487/?use_type=0&smp=1',
 'https://tabelog.com/oita/A4402/A440202/44005660/dtlrvwlst/B482850549/?use_type=0&smp=1',
 'https://tabelog.com/oita/A4402/A440202/44005660/dtlrvwlst/B494269956/?use_type=0&smp=1',
 'https://tabelog.com/oita/A4402/A440202/44005660/dtlrvwlst/B84684293/?use_type=0&smp=1'}

#### 2. 各レビューから取得したい内容のみを取得
- userid, name, rating, title, review
- 1つのURLでやってみる

In [16]:
rurl = reviewurls[0]
res = requests.get(rurl)
tree = html.fromstring(res.content)

In [17]:
#res.content

In [18]:
# User id/ Name
eles = tree.xpath("//a[@class='rvw-item__rvwr-name']")

In [19]:
eles[0].get("href")

'/rvwr/025510967/'

In [20]:
eles[0].getchildren()[0].text

'べっぴぃえんちゃん'

In [21]:
# rating
eles = tree.xpath("//div[@class='rvw-item__rvw-info']//b")

In [22]:
eles[0].text

'4.0'

In [23]:
#title
eles = tree.xpath("//p[@class='rvw-item__title']/strong")

In [24]:
eles[0].text

'いつも新たな出会いのあるお店'

In [25]:
# review
#eles = tree.xpath("//div[@class='rvw-item__rvw-comment']")
#eles = tree.xpath("//div[contains(@class, 'rvw-item') and contains(@class, 'rvw-comment')]")
eles = tree.xpath("//div[contains(@class, 'rvw-comment')]/p")

In [26]:
eles

[<Element p at 0xffff783a4e10>, <Element p at 0xffff783a4dc0>]

In [27]:
rawreview = eles[0].xpath("string()")

In [28]:
rawreview

'\n          3ヶ月ぶりの『トムヤムクン。』。お伺いするのは5、6回目だと思うが、来る度に新しい料理との出会いがある。メニューがなく、その時々の旬のもの、美味しいものを出してくれるシェフのおかげだ。アペリティフからして超独創的。こんな冷麺、見たことも食べたこともない。アジ、サバ、アサリの出汁に、新生姜とガスパチョ風のキュウリ。これだけでも斬新だが、ホタテの醤油漬けと一緒に食べると、あら不思議。塩ラーメンが一転醤油ラーメンに。本当に初めての味わいだ。次はクロダイのサラダ。コリンキーやズッキーニ、トマトはピクルスに。そして水菜、パクチーなど他の野菜やクロダイと一緒にキウイフルーツのソースで食べる。こんな発想聞いたことがない。魚は、一皿めはヘダイ。揚げたヘダイにとうもろこしのソース。とうもろこしが甘い。二皿めはイトヨリダイ。1番下に新タマネギのシャリアピンソース。魚の上には細かく刻んだナスとつるむらさき。オレガノも載っている。おそらくこれが味の決め手だろう。最後になって漸くタイ料理らしくなってきた（笑）まず看板料理のプーパッポンカリー。〆はエビのレッドカレー。レアなエビが美味しい。個人的にはもう少し辛くても良いが。まだまだ食べ足りない気もするが、ワインも空いたことだし、今日はここまで。ご馳走さまでした。今度は7皿でお願いするぞ〜\n        '

In [29]:
review = rawreview.strip()

In [30]:
# ここまでの内容を関数化
def scraper(rurl):
    res = requests.get(rurl)
    tree = html.fromstring(res.content)
    # User id/ Name
    eles = tree.xpath("//a[@class='rvw-item__rvwr-name']")
    uid = eles[0].get("href")
    name = eles[0].getchildren()[0].text
    # rating
    eles = tree.xpath("//div[@class='rvw-item__rvw-info']//b")
    rating = eles[0].text
    #title
    eles = tree.xpath("//p[@class='rvw-item__title']/strong")
    title = eles[0].text
    # review
    #eles = tree.xpath("//div[@class='rvw-item__rvw-comment']")
    #eles = tree.xpath("//div[contains(@class, 'rvw-item') and contains(@class, 'rvw-comment')]")
    eles = tree.xpath("//div[contains(@class, 'rvw-comment')]/p")
    rawreview = eles[0].xpath("string()")
    review = rawreview.strip()
    
    return uid, name, rating, title, review

In [31]:
rows = [["uid","name","rating", "title", "review"]]
for rurl in list(set(reviewurls)):
    row = scraper(rurl)
    rows.append(row)

In [32]:
import pandas as pd
pd.options.display.max_colwidth = 250

df = pd.DataFrame(rows[1:],columns=rows[0])

In [33]:
df

,uid,name,rating,title,review
0,/rvwr/025510967/,べっぴぃえんちゃん,4.0,いつも新たな出会いのあるお店,3ヶ月ぶりの『トムヤムクン。』。お伺いするのは5、6回目だと思うが、来る度に新しい料理との出会いがある。メニューがなく、その時々の旬のもの、美味しいものを出してくれるシェフのおかげだ。アペリティフからして超独創的。こんな冷麺、見たことも食べたこともない。アジ、サバ、アサリの出汁に、新生姜とガスパチョ風のキュウリ。これだけでも斬新だが、ホタテの醤油漬けと一緒に食べると、あら不思議。塩ラーメンが一転醤油ラーメンに。本当に初めての味わいだ。次はクロダイのサラダ。コリンキーやズッキーニ、トマトは...
1,/rvwr/000617475/,☆中隊長☆,3.9,【泰上々颱風】都会よりも洗練されたモダンタイランドは湯の街に♪,2021年長月だいぶ昔に行った時とは、完全にスタイルが変わってしまったコチラのお店！良い意味で洗練された料理を提供で【1日/1組のみの完全予約制】！？興味ワキワキだけどなかなか行く機会無かったが偶然の賜物で行けたよwwそして無茶ぶりの被害者がまた出た模様だけど！？（謎）【ディナーコースSPver(10000円）】※事前に食材をグレードアップ。〇長崎県産灸鰹の塩タタキと大分県産地野菜のピクルス人参のドレッシング様〇大分和牛もも肉のラープヌア大分県産雲丹乗せ様〇高知県産ノドグロの白ワイン蒸し...
2,/rvwr/001861477/,ぴんころ地蔵,5.0,別府の鬼才 トムヤンクン＼(^o^)／,さて、トムヤンクンさん。この日は奈々人さん夫妻と友人と4人で訪問。にこやかにマスターが迎え入れてくださる。この日は6品のコースです。前回同様独創力に長けたタイ料理。そこに様々な食の遊びを施すのがトムヤンクン流。 食材のコンビネーション、調味料。おっ！と刮目する料理がコースの中に散りばめられている。 こういう料理が別府で食べられることが奇跡であり、幸せでもある。また来年も１つよろしくお願いします。総括別府の鬼才 トムヤンクン
3,/rvwr/polpol/,polpol,4.5,別府でタイヌーベルキューイジーヌ、素晴らしい品々を,￼マイル消化で夫婦1泊2日の大分旅行を決め、別府温泉へ￼別府温泉には約50年ぶりの訪問。宿泊は温泉旅館￼だったので、晩御飯は旅館にて。と言うわけで、お昼に何かおいしいものを食べようと検索、￼こちらのお店に巡り会いました。大分の郷土料理も良いのですが、タイ料理をしばらく食べてなかったなぁと思ったので。レビューを読むと、シェフお1人で切り盛りされているとの事。￼先月1日1組の向ヶ丘遊園のイタリアンに行ったばかり、￼その魅力（シェフの考える世界の具現化）を知ってしまったので、とても興味を持ち予...
4,/rvwr/003971849/,しばきち,3.8,別府のタイ料理店。日本人向けのタイ料理を堪能出来て満足。,大分県別府市浜脇にあるタイ料理専門店。以前から行きたかったお店でようやく初訪問。完全予約制との事だったので事前に予約してからの訪問。店主は日本人。別府は多国籍な料理が多くその国の人が経営してることが多い。なので、こういうお店で日本人の店主というのは珍しく感じるね。色々なタイ料理を注文してシェア。中でも気に入ったのはガパオライスと海老の入ったタイ風焼飯。ガパオライスはナンプラーの風味も良くスパイシーな感じはあまりなくマイルド。挽肉はたっぷり。肉感が十分に感じられて美味しい。日本向けのガパオ...
5,/rvwr/000057392/,fujimo123,4.0,湯の街に佇む本場以上にハイレベルなタイ料理,理別府に来てます。今宵ディナーはこちらで予約。地元人でも有名な浜脇温泉地区に位置する料理タイ料理専門店。オーナーが1人で切り盛りしているため1日昼・夜各１組限定の完全予約制なので、今では2～3か月先しか取れないほど人気。メニューはお任せコースのみで、料金は品数で変わるシステム。再訪で今回も5品（４０００円）と、スパークリングワイン銘柄お任せでフルボトル１本を４人でシェアー。（計５５００円強）店内は１卓のみのこじんまりも相変わらず。まずはスパークリングワインで喉を潤してからコーススタート。...
6,/rvwr/010815537/,ALUCARD,4.5,今月２度目です,今夜は７皿の料理でオーダー昼からウイスキー空にしてたのに元気よかった＾＾
7,/rvwr/kumachelin/,クマシュラン,4.6,ヌーベルタイジーヌ,福岡の天ぷら屋の大将に勧められて訪問。一日に昼か夜の一組。これはタイ料理なのか？と驚く素晴らしい繊細な料理ばかり。シェフも楽しい方で、色々お話していただけた。旨くて楽しくて、また絶対に行きたいお店。
8,/rvwr/019641192/,かめすけもち,5.0,ワインもおすすめ！料理は当たり前においしい！！,1年ぶり3回目です。今回の九州旅行の目的のお店です。自分が目的の料理のために別府まで足を伸ばすなんて、と驚きつつ、やっぱりお邪魔してしまいます。今回はワインも堪能したいので、ホテルは駅近にしました。温泉に浸かりつつまったりしてしまい、ホテルに戻らずそのまま、入店です。すいません、すっぴんでとお詫びしつつ、さっぱりして、おいしいご飯です。ビール、ロゼ、赤のボトルと頂きつつ、毎回びっくりするお味とオーナーのトークです。今回はオーナーも慣れてくださったのか？トークもノリノリでした。お料理はビジ...
9,/rvwr/005332034/,フラワ田,3.9,サスティナブルなスタイル,・冷麺 海老の頭や鰹、浅利などから出汁原木椎茸は含め煮に つるむらさき 木耳甘いトマトのピューレ・クログジ 3日熟成、あかもく野菜のコンビネーションサラダ新生姜、コリンキーのソース・ガパオライスコロッケ黄身ソース、海老のトマトソース・ヤマトカマス 玉蜀黍 ココナッツミルクのピューレ、里芋の揚げ餅・ソフトシェルクラブのプーパッポンカリー・2日マリネしたみつせ鶏のコンフィ青唐辛子のハリッサとミントのソース・グリーンカレー サザエの肝ソースのリゾット ゴールデンフェニックス新牛蒡とサザエ メバ...


### BlueSkyデータの取得（API）

#### APIから投稿してみる

In [34]:
from atproto import Client
import pandas as pd

In [35]:
import json

with open("bsky_account.json", 'r') as f:
    acdic = json.load(f)

In [36]:
client = Client(base_url='https://bsky.social')
client.login(acdic["account"], acdic["password"])

ProfileViewDetailed(did='did:plc:7sqtnkepsjplcvuqpvprsx52', handle='bcoredsclass0001.bsky.social', associated=ProfileAssociated(chat=None, feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:7sqtnkepsjplcvuqpvprsx52/bafkreif4w23vqx2keibm2z665tm4i77kewnamclqtompktx6z6acdl2jdq@jpeg', banner=None, created_at='2025-04-10T00:59:16.772Z', description=None, display_name='', followers_count=1, follows_count=1, indexed_at='2025-04-10T00:59:16.772Z', joined_via_starter_pack=None, labels=[], pinned_post=None, posts_count=4, viewer=ViewerState(blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following=None, known_followers=None, muted=False, muted_by_list=None, py_type='app.bsky.actor.defs#viewerState'), py_type='app.bsky.actor.defs#profileViewDetailed')

In [37]:
client.send_post("今日はB-Coreで授業です")

CreateRecordResponse(uri='at://did:plc:7sqtnkepsjplcvuqpvprsx52/app.bsky.feed.post/3lo3zdhbhii24', cid='bafyreiayigcsnwc47cptlrb24du2xcilcsashtddxc3vzik3an33asghou')

#### キーワード検索

In [38]:
res = client.app.bsky.feed.search_posts({"q":"ミャクミャク", "sort":"top", "limit":100, "cursor":None})

In [39]:
print(res.cursor)
print(res.hits_total)

rows = [["name","text"]]
for post in res.posts:
#    print(post.cid)
#    print(post.author.did)
#    print(post.author.handle)
#    print(post.author.display_name)
#    print(post.author.avatar)
#    print(post.record.text)
#    print(post.like_count)
#    print(post.quote_count)
#    print(post.reply_count)
#    print(post.repost_count)
#    print(post.uri)
    name = post.author.display_name
    text = post.record.text.strip()
    row = [name, text]
    rows.append(row)

100
None


In [40]:
df = pd.DataFrame(rows[1:], columns=rows[0])

In [41]:
df

,name,text
0,へぼらん,キモかわいいやつ買った\n𐔌ᵔ｡◎⊙‎ࠏ⦿‪° ꒱\n#万博 #ミャクミャク
1,いしのこ,されちゃったの！？「寄生」！！\n\nクラフトホリック×ミャクミャク様コラボのスロース（くま）\nまさか岡山の店舗で手に入るとは思っていなかったので、お迎えできて嬉しい😆\nちなみに万博会場限定のぬいはTシャツを来てるみたいです👕\n\n#ぬいぐるみ #クラフトホリック\n#スロース #ミャクミャク #万博\n#ミャクミャクなりきりマスコットチャーム
2,鹿野ケル_Skeb募集中,ミャクミャクぽい子\n#illustration #イラスト
3,Sheep🐑,つい先程酔っぱらいながら描いたミャクミャク擬人化♀\n#イラスト #大阪・関西万博 #ミャクミャク
4,mimi🌷,ミャクミャク焼きついに！
...,...,...
95,蓮花茶,昨日、長男氏が万博行ってきたんだけど、噂の数百人待ちのスシローでお寿司食べてきたそう。整理券だけ午前中にもらって午後に行ったら、整理券さえあれば順番無しで入れてくれた模様（来ない人もいっぱいいるんだろうね）\nパピリオン予約は全滅だったけど、予約なしの海外パビリオンを結構堪能してきたみたい。もともと海外情報大好きな人なので万博楽しかったらしい\nそしてマシュマロ付きのミャクミャク貯金箱買ってきたんだけどそこそこ大きくて、どこに置くんだこれ…と家族会議中ｗ
96,トミーサンダー,おかしのまちおか、ローソン、ファミマ、セブンと回ったけど、ちいかわロリポップなかった…🍭\nその代わりと言っては何やけど、探しても見つからなかった貯金箱見つけた‼️\nしかもちいかわとハチワレが来ました☺️\n事務所に置いてスタッフに募金してもらおう😏\n\nついでに地元で開催されてるマスコットキャラのミャクミャクもいたので撮りました📷
97,hoshi,万博の展示とかにはあまり興味が無いもののミャクミャクのぬいがほしくて……
98,マ ス ター・ジョン ソン・ワッショイ,ミャクミャクのビロビロって触るとペットリしてそう


### ES基盤から検索してみる（研究室のサーバに負荷がかかるので注意）

In [42]:
from elasticsearch import Elasticsearch
import datetime as dt
from dateutil.relativedelta import relativedelta

In [43]:
# Elasticsearch の接続先
es = Elasticsearch("http://cicero.csis.oita-u.ac.jp:9200")

In [76]:
def get_post_uris(query, start, end):
    # クエリ条件に合致する投稿を取得（例では commit.record.text に対して match クエリ）
    body = {
        "size": 1000,  # 必要件数に応じて調整
        "timeout": "1m",
        "query": {
            "bool": {
                "must": [
                    {"match": {"commit.record.text": query}},
                    {"range": {"commit.record.createdAt": {"gte": start, "lte": end}}}
                ]
            }
        }
    }
    res =  es.options(request_timeout=60).search(index="postindex-*", body=body)
    posts = []  # 各投稿の情報（URI やテキストなど）
    uris = []   # 再構築した URI のリスト
    for hit in res["hits"]["hits"]:
        src = hit["_source"]
        try:
            did  = src["did"]
            coll = src["commit"]["collection"]
            rkey = src["commit"]["rkey"]
            # URI を "at://{did}/{collection}/{rkey}" として再構築
            uri = f"at://{did}/{coll}/{rkey}"
            uris.append(uri)
            # ここではテキスト（全文の一部など）も取得しておく（任意）
            text = src["commit"]["record"].get("text", "")
            posts.append({"uri": uri, "text": text})
        except Exception as e:
            continue
    return posts, uris

In [77]:
def get_like_ranking(query, start, end):
    posts, uris = get_post_uris(query, start, end)
    if not uris:
        return pd.DataFrame()

    body = {
        "size": 0,
        "timeout": "1m",
        "query": {
            "bool": {
                "filter": [
                    {"terms": {"commit.record.subject.uri": uris}},
                    {"term": {"commit.operation": "create"}},
                    {"range": {"commit.record.createdAt": {"gte": start, "lte": end}}}
                ]
            }
        },
        "aggs": {
            "post_likes": {
                "terms": {
                    "field": "commit.record.subject.uri",
                    "size": len(uris)
                }
            }
        }
    }
    res = es.options(request_timeout=60).search(index="likeindex-*", body=body)
    buckets = res["aggregations"]["post_likes"]["buckets"]
    df_aggs = pd.DataFrame([{"uri": b["key"], "like_count": b["doc_count"]} for b in buckets])
    posts_df = pd.DataFrame(posts)
#    st.dataframe(posts_df)
    merged = pd.merge(posts_df, df_aggs, on="uri", how="left").fillna(0)
    merged["like_count"] = merged["like_count"].astype(int)
    merged = merged.sort_values("like_count", ascending=False)
    return merged

In [78]:
today = dt.date.today()

In [79]:
start_date = today + relativedelta(days=-2)

In [80]:
start_date

datetime.date(2025, 4, 29)

In [81]:
start_date_str = start_date.strftime("%Y-%m-%dT00:00:00Z")

In [82]:
start_date_str

'2025-04-29T00:00:00Z'

In [83]:
end_date_str = today.strftime("%Y-%m-%dT23:59:59Z")

In [84]:
df = get_like_ranking(query="ミャクミャク", start=start_date_str, end=end_date_str)

In [85]:
df

,uri,text,like_count
79,at://did:plc:vtfpzbaahklcnmesip25tbam/app.bsky.feed.post/3lnxdzx47oc2v,されちゃったの！？「寄生」！！\n\nクラフトホリック×ミャクミャク様コラボのスロース（くま）\nまさか岡山の店舗で手に入るとは思っていなかったので、お迎えできて嬉しい😆\nちなみに万博会場限定のぬいはTシャツを来てるみたいです👕\n\n#ぬいぐるみ #クラフトホリック\n#スロース #ミャクミャク #万博\n#ミャクミャクなりきりマスコットチャーム,62
173,at://did:plc:3xyvfyoe56cqmnizvz5cr27l/app.bsky.feed.post/3lo2xajtd7c25,\ Hello ♡𝑴𝒂𝒚🎏/\n\n今日から5月ですね～✿\nはやっ( °_° )\n\n4月もたくさんのご予約\n本当にありがとうございました😘\n\n本日は15:00-21:30\n☞☞17:30～ご案内🉑です🤍ྀི\n\nご予約、お問い合わせ✉️\nお待ちしておりますね‪·͜·♡\n\nゴールデンウィーク\n皆様は何する予定❓\n\n私はとりあえず万博のID登録しました😊ミャクミャクのくじびきもしに行かなきゃな～🎶,28
41,at://did:plc:ioa6s6toxsiu3j6d5gtrgwue/app.bsky.feed.post/3lnzjyphzxc2h,キモかわいいやつ買った\n𐔌ᵔ｡◎⊙‎ࠏ⦿‪° ꒱\n#万博 #ミャクミャク,28
15,at://did:plc:n73azim7irnlcxibu5a5j5jk/app.bsky.feed.post/3lnwxbxygbc2x,ミャクミャク焼きですよ、ミミ…,18
163,at://did:plc:k3mvjdukr66iasname4jtj5t/app.bsky.feed.post/3lo2x34ewvk2t,おはようございます🌞\n今日から5月！\nじわじわと、関西万博に行きたくなってきた。これまでは興味なかったのにね、行かなかったらきっと50年は後悔しそう。\n\nさて、このアゲートの中にミャクミャクは隠れているでしょうか？\n\nブラジル産 アゲート\n\n#Bluesky鉱物部,17
...,...,...,...
116,at://did:plc:2c2w4ckipog5uhqbynnfr5ix/app.bsky.feed.post/3lnwp2ebkfc2f,期待はずれてちゃんとミャクミャク。\nただ、きんつばで再現する必要があったのかってぐらいわかりづらい,0
63,at://did:plc:4rffyanupza3s5nxz6t3rvm4/app.bsky.feed.post/3lnzbsos67f2p,鬼とミャクミャクが共演するGW夜の異色イベントを見逃すな！ #ミャクミャク #大江山酒呑童子 #鬼のまち\n\n2025年5月3日、京都府福知山市で開催される「大江山酒呑百鬼夜行」。万博公式キャラクター・ミャクミャクと鬼たちの共演が実現！,0
64,at://did:plc:lrslnrarlaszeil7zjw5enfd/app.bsky.feed.post/3lo3awkhqm224,好好笑世界觀混亂😍\nミャクミャク對霍格華茲來說顏色太飽和了！,0
113,at://did:plc:2eo6cpmgps6mmweul56vsyab/app.bsky.feed.post/3lnwolza2w22k,いやほんまにミャクミャクくじのお得さ、ぬいぐるみの良さをわかってくれる人おってよかった,0
